In [10]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense
import pyswarms as ps

# Load the dataset
df = pd.read_excel(r'D:/paper/qb/data/lab_data.xlsx')
df[1:100]
X = df.loc[:, ['H', 'S', 'Q', 'D50', 'D84', 'R']].values
y = df.loc[:, ['qs']].values

# Perform feature scaling
from sklearn.preprocessing import MinMaxScaler
scaler1 = MinMaxScaler().fit(X)
X1 = scaler1.transform(X)
scaler2 = MinMaxScaler().fit(y)
y1 = scaler2.transform(y)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X1, y1, test_size=0.2, random_state=0)
y_train = y_train.ravel()
y_test = y_test.ravel()

# Define the objective function to optimize
def optimize_model(particles):
    particles = particles.astype(int)

    # Define the model architecture
    model = Sequential()
    model.add(Dense(units=particles[:, 0], activation='relu', input_dim=X.shape[1]))
    model.add(Dense(units=particles[:, 1], activation='relu'))
    model.add(Dense(units=particles[:, 2], activation='relu'))
    model.add(Dense(units=1, activation='linear'))

    # Compile the model
    from tensorflow.keras.metrics import RootMeanSquaredError
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=[RootMeanSquaredError()])

    # Train the model
    model.fit(X_train, y_train, epochs=10, batch_size=10, verbose=0)

    # Evaluate the model
    loss = model.evaluate(X_test, y_test, verbose=0)[:, 0]

    return loss

# Define the bounds for particle values
bounds = (np.array([10, 10, 10]), np.array([30, 30, 30]))

# Define the PSO optimization function
def pso_optimization():
    # Define the PSO hyperparameters
    num_particles = 5
    max_iterations = 10

    # Define the options for the optimizer
    options = {'c1': 0.5, 'c2': 0.3, 'w': 0.9}

    # Initialize the optimizer
    optimizer = ps.single.GlobalBestPSO(n_particles=num_particles, dimensions=3, bounds=bounds, options=options)

    # Perform optimization
    best_cost, best_pos = optimizer.optimize(optimize_model, iters=max_iterations, n_processes=4)

    # Print the best cost and particle values
    print('Best Cost:', best_cost)
    print('Best Particle:', best_pos)

# Run the PSO optimization
pso_optimization()


2023-06-25 22:54:05,303 - pyswarms.single.global_best - INFO - Optimize for 10 iters with {'c1': 0.5, 'c2': 0.3, 'w': 0.9}
pyswarms.single.global_best:   0%|          |0/10